In [9]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from logging import info, debug
from tkinter import W
import numpy as np
import os
import logging
import numpy as np
from tqdm import tqdm
import matplotlib
from PIL import Image
from skimage import io
import pprint
from models.utils import show_progress
import numpy as np
from logging import log, info, debug
from PIL import Image
import tensorflow as tf
import numpy as np
import pandas as pd
import cv2 as cv
import sys
from models.models import build_resnet, build_siamese_autoencoder
from losses import DiceLoss
from metrics import DiceMetric
from matplotlib import pyplot as plt

sys.path.append('/home/aadi/projects/pgrad-thesis/data')

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [29]:
def _normalize(img):
    img = tf.cast(img, tf.float32) / 255.0
    return img


def decode_grey(img):
    img = tf.io.decode_png(img, channels=1)
    return img


def decode_rgb(img):
    img = tf.io.decode_png(img, channels=3)
    return img


def process_path_rgb(fp):
    print(fp)
    img = tf.io.read_file(fp)
    img = decode_rgb(img)
    img = _normalize(img)
    return img


def process_path_grey(fp):
    img = tf.io.read_file(fp)
    img = decode_grey(img)
    return img


def build_data_horizontal_separate(batch_size, take, buffer_size=1000):

    def train_gen(split='train', data_path='../data/'):
        path = data_path + split
        for t1, t2, l in zip(sorted(os.listdir(path+'/time1')), sorted(os.listdir(path+'/time2')), sorted(os.listdir(path+'/label'))):
            # get full paths

            t1 = process_path_rgb(f'../data/{split}/time1/' + t1)
            t2 = process_path_rgb(f'../data/{split}/time2/' + t2)
            l = process_path_grey(f'../data/{split}/label/' + l)

            yield (t1, t2), l

    def val_gen(split='val', data_path='../data/'):
        path = data_path + split
        for t1, t2, l in zip(sorted(os.listdir(path+'/time1')), sorted(os.listdir(path+'/time2')), sorted(os.listdir(path+'/label'))):
            # get full paths

            t1 = process_path_rgb(f'data/{split}/time1/' + t1)
            t2 = process_path_rgb(f'data/{split}/time2/' + t2)
            l = process_path_grey(f'data/{split}/label/' + l)

            yield (t1, t2), l

    train_ds = tf.data.Dataset.from_generator(
        train_gen, output_types=((tf.float32, tf.float32), tf.uint8))
    val_ds = tf.data.Dataset.from_generator(
        val_gen, output_types=((tf.float32, tf.float32), tf.uint8))

    train_batches = (
        train_ds
        .cache()
        .shuffle(buffer_size)
        .batch(batch_size)
        .take(take)
        .prefetch(buffer_size=tf.data.AUTOTUNE))

    val_batches = (
        val_ds
        .cache()
        .shuffle(buffer_size)
        .batch(batch_size)
        .take(take)
        .prefetch(buffer_size=tf.data.AUTOTUNE))

    return train_batches, val_batches

In [34]:
train, val = build_data_horizontal_separate(batch_size=25, take=100)

In [36]:
model

NameError: name 'model' is not defined

../data/train/time1/00000.png
../data/train/time1/00000.png
../data/train/time2/00000.png


((<tf.Tensor: shape=(256, 256, 3), dtype=float32, numpy=
  array([[[0.18039216, 0.36862746, 0.27450982],
          [0.08235294, 0.27058825, 0.1764706 ],
          [0.16078432, 0.34901962, 0.25490198],
          ...,
          [0.23137255, 0.40392157, 0.2784314 ],
          [0.33333334, 0.49411765, 0.3647059 ],
          [0.30588236, 0.5176471 , 0.37254903]],
  
         [[0.09803922, 0.28627452, 0.20392157],
          [0.13725491, 0.3254902 , 0.23137255],
          [0.15686275, 0.34509805, 0.24313726],
          ...,
          [0.18039216, 0.36862746, 0.22352941],
          [0.36078432, 0.49803922, 0.36078432],
          [0.28235295, 0.49411765, 0.34117648]],
  
         [[0.06666667, 0.25490198, 0.16078432],
          [0.12156863, 0.30980393, 0.20784314],
          [0.14509805, 0.33333334, 0.23137255],
          ...,
          [0.14901961, 0.34509805, 0.20392157],
          [0.28627452, 0.45490196, 0.31764707],
          [0.36862746, 0.5803922 , 0.43529412]],
  
         ...,
  
     